In [11]:
import re

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from nltk import accuracy
from sklearn import datasets
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch import dtype
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from BayesianLLNN import BayesianLastLayer,PredLastLayer,TrainLastLayer
from baseNN import GetNeuralNetwork, PredNormal, TrainNN, GetNeuralNetworkClassification, TrainNNClass, PredictClass

Loading Dataset

In [12]:
np.random.seed(123)
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

In [13]:
# load dataset, medical, diabetes, relevant. Diagnosis
df = pd.read_csv("/Users/kingmopser/PycharmProjects/ProbabilisticML/Dataset of Diabetes .csv")
df.drop(columns=["ID","No_Pation"], inplace=True)
len(df.columns) # 13

12

Pre-Processing for Input Data

In [14]:
scaler = StandardScaler()
encoder =LabelEncoder()
df["Gender"] = df["Gender"].apply(lambda x: 1 if x=="M" else 0)
c = df.columns[1:len(df.columns)-1]
df[c] =scaler.fit_transform(df[c])
df["CLASS"] = df["CLASS"].apply(lambda x: "N" if re.match(r"N",x) else "P" if x=="P" else "Y" if re.match(r"Y",x) else 0)
df["CLASS"]= pd.Categorical(df["CLASS"], categories=["N","P","Y"], ordered=True)
df["CLASS"] = encoder.fit_transform(df["CLASS"])

In [15]:
X = df.iloc[:,:-1].to_numpy(dtype=np.float32)
y= df['CLASS'].to_numpy()
X_train,X_test,y_train,y_test = train_test_split(X,y, train_size=0.8)
tenX = torch.tensor(X_train).to(device)
tenY = torch.tensor(y_train).to(device)
tenXTest = torch.tensor(X_test).to(device)
tenYTest = torch.tensor(y_test).to(device)
tenDF = TensorDataset(tenX,tenY)
trainLoader = DataLoader(tenDF,batch_size=32,shuffle=True)
#IND and OOD



Fitting Models

In [16]:
# BASE NN CLASSIFIER
nn = GetNeuralNetworkClassification(len(df.columns[:-1]),len(np.unique(y)))
TrainNNClass(nn,trainLoader,epochs=100)
cl = PredictClass(nn,tenXTest)

Epoch: 0 | Loss: 0.83571
Epoch: 10 | Loss: 0.24323
Epoch: 20 | Loss: 0.04310
Epoch: 30 | Loss: 0.14292
Epoch: 40 | Loss: 0.05820
Epoch: 50 | Loss: 0.04691
Epoch: 60 | Loss: 0.04011
Epoch: 70 | Loss: 0.02746
Epoch: 80 | Loss: 0.00518
Epoch: 90 | Loss: 0.00769


In [21]:
#Metrics





Epoch: 0 | Loss: 0.81997
Epoch: 10 | Loss: 0.06064
Epoch: 20 | Loss: 0.11965
Epoch: 30 | Loss: 0.02723
Epoch: 40 | Loss: 0.10717
